<a href="https://colab.research.google.com/github/fmarcelneves/Twitter-Network-Analytics/blob/main/Machine_learning_model_construction_Felipe_M_Neves_political_theme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Portuguese Tweets for Sentiment Analysis using nltk and sklearn</h1>

## Install package

In [4]:
!pip install xgboost

## Load packages

In [5]:
import os
import pandas as pd
import numpy as np
import random
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
# from nltk.tokenize import sent_tokenize (Tokenization)
from nltk.probability import FreqDist
from nltk.metrics import ConfusionMatrix
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
import  xgboost as xgb

In [6]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
%%capture
!pip install plotly --upgrade
from plotly import __version__
print (__version__)
import plotly.express as px
import plotly as py
#import plotly.figure_factory as ff
#import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
import plotly.io as pio
pio.templates.default = "plotly_white"

## Load and concatenate dataset

<h4>Load and prepare</h4>

In [8]:
TweetsNeutralHash = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/TweetsNeutralHash.csv'
TweetsNeutralNews = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/TweetsNeutralNews.csv'
TweetsWithTheme = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/TweetsWithTheme.csv'
#Train3Classes = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/Train3Classes.csv'
NoThemeTweets = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/NoThemeTweets.csv'

In [9]:
NeutralHash = pd.read_csv(TweetsNeutralHash, encoding='utf-8')
NeutralNews = pd.read_csv(TweetsNeutralNews, encoding='utf-8')
Political_Neg_Pos = pd.read_csv(TweetsWithTheme, encoding='utf-8')
#Classes = pd.read_csv(Train3Classes, encoding='utf-8', sep=';')
NoThemeTweets_Pos = pd.read_csv(NoThemeTweets, encoding='latin-1', sep=';')

Neutral

In [10]:
NeutralHash  = NeutralHash[['tweet_text', 'sentiment']]
NeutralHash.columns = ['texto', 'classificacao']

In [11]:
NeutralHash.head()

,texto,classificacao
0,@dilsonramoslima #Fato Acho que o Roger é um b...,Neutro
1,#NOVIDADE! @LATAM_BRA acaba de anunciar novo v...,Neutro
2,Quando tem #novidade😆 tem @novafm103 na área! ...,Neutro
3,@RiodeNojeira #Novidade Taí o sucesso dos filh...,Neutro
4,"[Livro/Novidades] Segredos, uma história de Lu...",Neutro


In [12]:
NeutralHash = NeutralHash.drop_duplicates().reset_index(drop=True)

In [13]:
NeutralHash.shape

(15515, 2)

In [14]:
NeutralHash = NeutralHash.sample(8000) #14000 #8000

In [15]:
NeutralHash.shape

(8000, 2)

In [16]:
NeutralNews  = NeutralNews[['tweet_text', 'sentiment']]
NeutralNews.columns = ['texto', 'classificacao']

In [17]:
NeutralNews = NeutralNews.drop_duplicates().reset_index(drop=True)

In [18]:
NeutralNews.shape

(37509, 2)

In [19]:
NeutralNews = NeutralNews.sample(20000) #14000 #20000

In [20]:
NeutralNews.shape

(20000, 2)

In [21]:
#NeutralHash+NeutralNews = Neutral

Positive 

In [22]:
Political_Neg_Pos  = Political_Neg_Pos[['tweet_text', 'sentiment']]
Political_Neg_Pos.columns = ['texto', 'classificacao']

In [23]:
Political_Neg_Pos.shape

(61591, 2)

In [24]:
Political_Neg_Pos = Political_Neg_Pos.drop_duplicates().reset_index(drop=True)

In [25]:
Political_Neg_Pos.shape

(61283, 2)

In [26]:
Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Positivo'].shape

(32566, 2)

In [27]:
Political_Pos = Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Positivo'].sample(20000) #18000 #20000 #14000

In [28]:
Political_Pos.shape

(20000, 2)

In [29]:
NoThemeTweets_Pos.columns = ['texto', 'classificacao']

In [30]:
NoThemeTweets_Pos.head()

,texto,classificacao
0,@Tixaa23 14 para eu ir :),Positivo
1,@drexalvarez O meu like eu já dei na época :),Positivo
2,Eu só queria conseguir comer alguma coisa pra ...,Positivo
3,:D que lindo dia !,Positivo
4,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Positivo


In [31]:
NoThemeTweets_Pos = NoThemeTweets_Pos.drop_duplicates().reset_index(drop=True)

In [32]:
NoThemeTweets_Pos.shape

(19983, 2)

In [33]:
NoTheme_Pos = NoThemeTweets_Pos[NoThemeTweets_Pos['classificacao'] == 'Positivo'].sample(8000)

In [34]:
NoTheme_Pos.shape

(8000, 2)

In [35]:
#from IPython.display import HTML
#HTML(Political_Pos.to_html())

In [36]:
#Classes_Pos  = Classes[['tweet_text', 'sentiment']]
#Classes_Pos.columns = ['texto', 'classificacao']

In [37]:
#Classes_Pos['classificacao'] = Classes_Pos['classificacao'].map({0: 'Negativo', 1: 'Positivo', 2: 'Neutro'})

In [38]:
#Classes_Pos = Classes_Pos.loc[Classes_Pos['classificacao'] == 'Positivo'].sample(8000) #10000 #14000 #8000

In [39]:
#Classes_Pos.shape

In [40]:
#Political_Pos+NoTheme_Pos = Positive

Negative

In [41]:
Political_Neg_Pos = Political_Neg_Pos.drop_duplicates().reset_index(drop=True)

In [42]:
Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Negativo'].shape

(28717, 2)

In [43]:
Political_Neg = Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Negativo'].sample(28000)

In [44]:
Political_Neg.shape

(28000, 2)

In [45]:
dataset = pd.concat([NeutralHash, NeutralNews, Political_Pos, NoTheme_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
dataset

,texto,classificacao
0,Somente verdades! #Fato.😸😌 em Três Lagoas http...,Neutro
1,Os mais famosos são os maiores solitários. #Fato,Neutro
2,Aquela pausa pro café nosso de cada dia. Café ...,Neutro
3,"O ministro da Defesa, Joaquim Silva e Luna . B...",Neutro
4,#seveja #sevejacombr #paulovizeu #fotografia #...,Neutro
...,...,...
83919,@peekabruwu sim :(((((((((((( só o lula mesmo ...,Negativo
83920,"@diskluma dscp por isso luma, mas era o melhor...",Negativo
83921,@_btss_armyy_ @95anasantos @carolinavicenet @F...,Negativo
83922,@brunamoliveiraa pq vai ser no dia das eleiçõe...,Negativo


In [46]:
dataset.shape

(83924, 2)

In [47]:
#dataset = pd.concat([NeutralHash, NeutralNews, Political_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
#dataset

In [48]:
#dataset = pd.concat([NeutralNews, Political_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
#dataset

## Exploratory Analysis

In [49]:
dataset.sample(5)

,texto,classificacao
61685,@Lpersonal @Haddad_Fernando pedindo medidas dr...,Negativo
55232,natural quando faz merda perde o shape rapido ...,Positivo
20009,Drauzio Varella protagoniza web série sobre qu...,Neutro
65200,"Pt tenho o filme , tenho as pipocas só falta a...",Negativo
34478,"Gente, cada vez mais fácil votar nessas eleiçõ...",Positivo


In [50]:
series = dataset['classificacao'].value_counts()
series

Negativo    28000
Neutro      27991
Positivo    27933
Name: classificacao, dtype: int64

In [51]:
pd.options.plotting.backend = "plotly"

In [52]:
series.plot(kind='bar', title='Número de tweets classificados para cada sentimento considerado', 
labels={"value": "Frequências", "index": "Sentimentos"}, color = ['Negativo', 'Neutro', 'Positivo'], 
color_discrete_sequence =['#EF553B', '#00CC96', '#636EFA'])

## Text Preprocessing

  Noise Removal
    <ul>
      <li>Remove urls from text (http(s), www)</li>
      <li>Remove a regex pattern (hashtags, notations)</li>
      <li>Prepare text, replace emoticons ":)) :) :D :(" to positive_emoticon or negative_emoticon or neutral_emoticon tags</li>
  <li>Tokenize text with nltk</li>
  <li>Object Standardization</li>
  <li>Remove stopwords (noise removal and dimensionality reduction)</li>
  <li>Lexicon Normalization</li>
    <ul>
      <li>Lemmatization (not implemented...)</li>
      <li>Stemming (dimensionality reduction)</li>
    </ul>
</ul> 

<h4>Examples of emoticons</h4>

<h4>Noise Removal</h4>

Remove urls from text (http(s), www)

In [53]:
def _remove_url(data):
    ls = []
    words = ''
    regexp1 = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    regexp2 = re.compile('www?.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    
    for line in data:
        urls = regexp1.findall(line)

        for u in urls:
            line = line.replace(u, ' ')

        urls = regexp2.findall(line)

        for u in urls:
            line = line.replace(u, ' ')
            
        ls.append(line)
    return ls

In [54]:
dataset['texto'] = _remove_url(dataset['texto'])

Remove a regex pattern (hashtags, notations)

In [55]:
def _remove_regex(data, regex_pattern):
    ls = []
    words = ''
    
    for line in data:
        matches = re.finditer(regex_pattern, line)
        
        for m in matches: 
            line = re.sub(m.group().strip(), '', line)

        ls.append(line)

    return ls

In [56]:
# hashtags
regex_pattern = '#[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [57]:
# notations
regex_pattern = '@[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [58]:
# rt
regex_pattern = 'RT[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [ ]:
#numbers 
regex_pattern = '\d+'
sa_twitter_dt = _remove_regex(sa_twitter_dt, regex_pattern)

In [59]:
 emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

In [60]:
# emoji pattern 
regex_pattern = emoji_pattern 
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [61]:
# check data
for i in range(0, 5):
    print(dataset['texto'][i])

Somente verdades! . em Três Lagoas  
Os mais famosos são os maiores solitários. 
Aquela pausa pro café nosso de cada dia. Café é vida!!!!! 🧠🧐‍⏏🤳           Avante em…  
O ministro da Defesa, Joaquim Silva e Luna . Brasil e EUA voltam a tratar do uso de Alcântara para lançar foguetes . Leia mais em   (link na bio).  …  
 combr             -   em Prefeitura de Miguel Pereira  


Prepare text, replace emoticons ":)) :) :D :(" to positive_emoticon or negative_emoticon or neutral_emoticon

In [62]:
def _replace_emoticons(data, emoticon_list):
    ls = []

    for line in data:
        for exp in emoticon_list:
            line = line.replace(exp, emoticon_list[exp])

        ls.append(line)

    return ls

In [63]:
emoticon_list = {':))': 'emoticon_positivo', ':)': 'emoticon_positivo', ':D': 'emoticon_positivo', ':-)': 'emoticon_positivo',  ':(': 'emoticon_negativo', ':((': 'emoticon_negativo', ':-(' : 'emoticon_negativo', '8)': 'emoticon_neutro'}

In [64]:
dataset['texto']= _replace_emoticons(dataset['texto'], emoticon_list)

In [65]:
# check data
for i in range(0, 5):
    print(dataset['texto'][i])

Somente verdades! . em Três Lagoas  
Os mais famosos são os maiores solitários. 
Aquela pausa pro café nosso de cada dia. Café é vida!!!!! 🧠🧐‍⏏🤳           Avante em…  
O ministro da Defesa, Joaquim Silva e Luna . Brasil e EUA voltam a tratar do uso de Alcântara para lançar foguetes . Leia mais em   (link na bio).  …  
 combr             -   em Prefeitura de Miguel Pereira  


In [66]:
dataset

,texto,classificacao
0,Somente verdades! . em Três Lagoas,Neutro
1,Os mais famosos são os maiores solitários.,Neutro
2,Aquela pausa pro café nosso de cada dia. Café ...,Neutro
3,"O ministro da Defesa, Joaquim Silva e Luna . B...",Neutro
4,combr - em Prefeitura de Miguel...,Neutro
...,...,...
83919,sim emoticon_negativo((((((((((( só o lula me...,Negativo
83920,"dscp por isso luma, mas era o melhor pra econ...",Negativo
83921,Timida e não sei meter conversa e f...,Negativo
83922,"pq vai ser no dia das eleições, vai ta uma lo...",Negativo


In [67]:
nan_value = float("NaN")
dataset.replace("", nan_value, inplace=True)

In [69]:
dataset

,texto,classificacao
0,Somente verdades! . em Três Lagoas,Neutro
1,Os mais famosos são os maiores solitários.,Neutro
2,Aquela pausa pro café nosso de cada dia. Café ...,Neutro
3,"O ministro da Defesa, Joaquim Silva e Luna . B...",Neutro
4,combr - em Prefeitura de Miguel...,Neutro
...,...,...
83919,sim emoticon_negativo((((((((((( só o lula me...,Negativo
83920,"dscp por isso luma, mas era o melhor pra econ...",Negativo
83921,Timida e não sei meter conversa e f...,Negativo
83922,"pq vai ser no dia das eleições, vai ta uma lo...",Negativo


<h4>Tokenize text with nltk</h4>

In [70]:
def _tokenize_text(data):
    ls = []

    for line in data:
        tokens = wordpunct_tokenize(line)
        ls.append(tokens)

    return ls

In [71]:
dataset_texto_tokens = _tokenize_text(dataset['texto'])

In [72]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['Somente', 'verdades', '!', '.', 'em', 'Três', 'Lagoas']
['Os', 'mais', 'famosos', 'são', 'os', 'maiores', 'solitários', '.']
['Aquela', 'pausa', 'pro', 'café', 'nosso', 'de', 'cada', 'dia', '.', 'Café', 'é', 'vida', '!!!!!', '🧠🧐\u200d⏏🤳', 'Avante', 'em', '…']
['O', 'ministro', 'da', 'Defesa', ',', 'Joaquim', 'Silva', 'e', 'Luna', '.', 'Brasil', 'e', 'EUA', 'voltam', 'a', 'tratar', 'do', 'uso', 'de', 'Alcântara', 'para', 'lançar', 'foguetes', '.', 'Leia', 'mais', 'em', '(', 'link', 'na', 'bio', ').', '…']
['combr', '-', 'em', 'Prefeitura', 'de', 'Miguel', 'Pereira']


In [73]:
#dataset_texto_tokens = list(filter(None, dataset_texto_tokens))

In [74]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['Somente', 'verdades', '!', '.', 'em', 'Três', 'Lagoas']
['Os', 'mais', 'famosos', 'são', 'os', 'maiores', 'solitários', '.']
['Aquela', 'pausa', 'pro', 'café', 'nosso', 'de', 'cada', 'dia', '.', 'Café', 'é', 'vida', '!!!!!', '🧠🧐\u200d⏏🤳', 'Avante', 'em', '…']
['O', 'ministro', 'da', 'Defesa', ',', 'Joaquim', 'Silva', 'e', 'Luna', '.', 'Brasil', 'e', 'EUA', 'voltam', 'a', 'tratar', 'do', 'uso', 'de', 'Alcântara', 'para', 'lançar', 'foguetes', '.', 'Leia', 'mais', 'em', '(', 'link', 'na', 'bio', ').', '…']
['combr', '-', 'em', 'Prefeitura', 'de', 'Miguel', 'Pereira']


<h4>Object Standardization</h4>

In [75]:
def _apply_standardization(tokens, std_list):
    ls = []

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            if word.lower() in std_list:
                word = std_list[word.lower()]
                
            new_tokens.append(word) 
            
        ls.append(new_tokens)

    return ls

In [76]:
# create your own list
std_list = {'eh': 'é', 'vc': 'você', 'vcs': 'vocês','tb': 'também', 'tbm': 'também', 'obg': 'obrigado', 'gnt': 'gente', 'q': 'que', 'n': 'não', 'cmg': 'comigo', 'p': 'para', 'ta': 'está', 'to': 'estou', 'vdd': 'verdade'}

In [77]:
dataset_texto_tokens = _apply_standardization(dataset_texto_tokens, std_list)

In [78]:
# check data
print(dataset_texto_tokens[4], dataset_texto_tokens[1008])

['combr', '-', 'em', 'Prefeitura', 'de', 'Miguel', 'Pereira'] ['Confira', 'mais', 'sobre', 'o', 'ocorrido', '.']


<h4>Remove stopwords (noise removal and dimensionality reduction)</h4>

Remove stopwords

In [79]:
def _remove_stopwords(tokens, stopword_list):
    ls = []

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            if word.lower() not in stopword_list:
                new_tokens.append(word) 
            
        ls.append(new_tokens)
        
    return ls

In [80]:
stopword_list = []

In [81]:
# get nltk portuguese stopwords
nltk_stopwords = nltk.corpus.stopwords.words('portuguese')

In [82]:
# You can also add stopwords manually instead of loading from the database. Generally, we add stopwords that belong to this context.
stopword_list.append('é')
stopword_list.append('vou')
stopword_list.append('que')
stopword_list.append('tão')
stopword_list.append('...')
stopword_list.append('«')
stopword_list.append('➔')
stopword_list.append('|')
stopword_list.append('!!')
stopword_list.append('!!!')
stopword_list.append('!!!!')
stopword_list.append('//')
stopword_list.append('/')
stopword_list.append('||') 
stopword_list.append(':')
stopword_list.append('-')
stopword_list.append(';')
stopword_list.append('»')
stopword_list.append('…')
stopword_list.append(')')
stopword_list.append('(')
stopword_list.append('uai') 
stopword_list.append(']')
stopword_list.append('[')
stopword_list.append('[]')
stopword_list.append('((')
stopword_list.append('))')
stopword_list.append(',')
stopword_list.append(', ')
stopword_list.append('...')
stopword_list.append('.')
stopword_list.append('•')
stopword_list.append('$')
stopword_list.append('%')
stopword_list.append('()')
stopword_list.append(').')
stopword_list.append('..')
stopword_list.append('”')
stopword_list.append('“')
stopword_list.append("'")
stopword_list.append('=')
stopword_list.append('&')
stopword_list.append('\u200d')
stopword_list.append('“”')
stopword_list.append('[]')
stopword_list.append('^^')
stopword_list.append('(((((')
stopword_list.append('(((')
stopword_list.append(')))')
stopword_list.append(')))))')
stopword_list.append('emoticon_positivo') 
stopword_list.append('emoticon_negativo')
stopword_list.append('emoticon_neutro')

<h5 style="color:red;">Note: Numbers, some punctuations (.,;:) and special characters such as (_- = &%) are not relevant to express some sentiment, they can be considered noisy words. You can treat them here.</h5>
<h5 style="color:red;">noisy_words = ['.', '?', '!', ':', ',', ';', '(', ')', '-']</h5>
<h5 style="color:red;">Note: ? and ! can be important to express sentiment.</h5>

In [83]:
# join all stopwords
stopword_list.extend(nltk_stopwords)
#stopword_list.extend(custom_stopwords'\d+')

In [84]:
# remove duplicate stopwords (unique list)
stopword_list = list(set(stopword_list))

In [85]:
dataset_texto_tokens = _remove_stopwords(dataset_texto_tokens, stopword_list)

In [159]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

Somente verdades ! Três Lagoas 
famosos maiores solitários 
pausa pro café cada dia Café vida !!!!! 🧠🧐‍⏏🤳 Avante 
ministro Defesa Joaquim Silva Luna Brasil EUA voltam tratar uso Alcântara lançar foguetes Leia link bio 
combr Prefeitura Miguel Pereira 


<h4>Lexicon Normalization</h4>

Lemmatization (not implemented...)

Stemming (dimensionality reduction)

In [87]:
def _apply_stemmer(tokens):
    ls = []
    stemmer = nltk.stem.RSLPStemmer()

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            word = str(stemmer.stem(word))
            new_tokens.append(word) 
            
        ls.append(new_tokens)
        
    return ls

In [88]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [89]:
dataset_texto_stemm = _apply_stemmer(dataset_texto_tokens)

In [90]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['Somente', 'verdades', '!', 'Três', 'Lagoas']
['famosos', 'maiores', 'solitários']
['pausa', 'pro', 'café', 'cada', 'dia', 'Café', 'vida', '!!!!!', '🧠🧐\u200d⏏🤳', 'Avante']
['ministro', 'Defesa', 'Joaquim', 'Silva', 'Luna', 'Brasil', 'EUA', 'voltam', 'tratar', 'uso', 'Alcântara', 'lançar', 'foguetes', 'Leia', 'link', 'bio']
['combr', 'Prefeitura', 'Miguel', 'Pereira']


<h4>Dataset preparation</h4>
Untokenize text (transform tokenized text into string list)

In [91]:
def _untokenize_text(tokens):
    ls = []

    for tk_line in tokens:
        new_line = ''
        
        for word in tk_line:
            new_line += word + ' '
            
        ls.append(new_line)
        
    return ls

In [92]:
dataset_texto_tokens = _untokenize_text(dataset_texto_tokens)

In [93]:
dataset_texto_stemm = _untokenize_text(dataset_texto_stemm)

In [94]:
# check data
for i in range(0, 5):
    print(dataset_texto_stemm[i])

soment verdad ! trê lago 
fam mai solit 
paus pro café cad dia café vid !!!!! 🧠🧐‍⏏🤳 av 
ministr defês joaquim silv lun brasil eua volt trat uso alcânt lanç foguet lei link bio 
combr prefeit miguel per 


## Implementing a Model


In [95]:
X_tweets = np.array(dataset_texto_stemm,  dtype=object)
X_tweets

array(['soment verdad ! trê lago ', 'fam mai solit ',
       'paus pro café cad dia café vid !!!!! 🧠🧐\u200d⏏🤳 av ', ...,
       'tim sei met convers faç figur ot entant esper dia poss acontec esper estej bem send possível est bem sempr tenh sempr forç ergu lut felic esper estej feliz + ',
       'pq vai ser dia ele vai louc vai ser doming est aqu (((( ',
       'quer est ap menos sement dess pra esperanç morr oq acontec fortalec famíl déc monopoliz polít fic feliz temp trist '],
      dtype=object)

In [96]:
y_classes = dataset["classificacao"].values
y_classes

array(['Neutro', 'Neutro', 'Neutro', ..., 'Negativo', 'Negativo',
       'Negativo'], dtype=object)

In [97]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neural_network import MLPClassifier

Split data

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X_tweets, y_classes, test_size=0.33, random_state=42)

## Random Forest

In [ ]:
model = RandomForestClassifier(random_state=42, n_jobs=-1)

In [ ]:
model_RF = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
])

### Random forest model with Hold-out

In [ ]:
model_RF.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [ ]:
predictions_RF = model_RF.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_RF))

              precision    recall  f1-score   support

    Negativo       0.69      0.77      0.73      9256
      Neutro       0.86      0.81      0.83      9117
    Positivo       0.65      0.61      0.63      9244

    accuracy                           0.73     27617
   macro avg       0.73      0.73      0.73     27617
weighted avg       0.73      0.73      0.73     27617



In [ ]:
#for doc, prediction, y in zip(X_test[0:20], predictions_RF[0:20], y_test[0:20]):
#    print('%r => %s [%s]' % (doc, prediction, y))

## XGBoost 

In [134]:
model_XGB = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', xgb.XGBClassifier(n_estimators=1000, n_jobs=-1, nthread=-1))
])

##XGBoost with Hold-out

In [135]:
model_XGB.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing

In [136]:
predictions_XGB = model_XGB.predict(X_test)

In [137]:
print(classification_report(y_test, predictions_XGB))

              precision    recall  f1-score   support

    Negativo       0.71      0.74      0.72      9227
      Neutro       0.78      0.85      0.81      9124
    Positivo       0.67      0.58      0.62      9344

    accuracy                           0.72     27695
   macro avg       0.72      0.72      0.72     27695
weighted avg       0.72      0.72      0.72     27695



## Neural Network - Multilayer Perceptron

<img src="https://st2.ning.com/topology/rest/1.0/file/get/1140047167?profile=original" width="500"/>

In [130]:
model_NN = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('pca', TruncatedSVD(n_components=500)), 
    ('clf', MLPClassifier(hidden_layer_sizes=(100,50), #2 camadas escondidas sendo a primeira com 100 neurônios e a segunda com 50 neurônios 
                          random_state=42, 
                          activation='relu', #Try othe functions. Ex. tanh, relu...
                          max_iter=1000,
                          early_stopping=True))
])

In [143]:
#teste = CountVectorizer(ngram_range=(1, 2))
#teste.fit_transform(X_train)

### Neural network model with Hold-out

In [131]:
model_NN.fit(X_train, y_train)
predictions_NN = model_NN.predict(X_test)

In [132]:
print(classification_report(y_test, predictions_NN))

              precision    recall  f1-score   support

    Negativo       0.72      0.75      0.73      9227
      Neutro       0.85      0.83      0.84      9124
    Positivo       0.65      0.65      0.65      9344

    accuracy                           0.74     27695
   macro avg       0.74      0.74      0.74     27695
weighted avg       0.74      0.74      0.74     27695



In [109]:
for doc, prediction, y in zip(X_test[0:20], predictions_NN[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'nunc peg ranç tant gent famíl rápid ness ele ' => Negativo [Positivo]
'supost pesquis ? tant quant sab veríd ach legal invert tud manch candidat link mesm coloc desm fal víde menin ' => Positivo [Positivo]
'áudi dela reforç tes odebrecht deu dinh amig tem diz pf lei transcr ' => Neutro [Neutro]
'escolh deput barr projet presid ? khkjkjkjk tá difícil polít ' => Positivo [Negativo]
'conf seleç tapet dar vid complement ambi cas ' => Neutro [Neutro]
'marin fal alg ? vi ' => Negativo [Negativo]
'‘ fal beste sim moment raiv separ ’, diz ex mulh bolsonar ' => Positivo [Neutro]
'vot vam vot ' => Positivo [Positivo]
'candidat ? — vot tô tailând ' => Negativo [Negativo]
'nezer pûç be ' => Positivo [Positivo]
'exat entend ministr fazend vai ser paul gued ach lul entend econom ? kkk ' => Positivo [Positivo]
'profes liset candidat psol govern sp defend discuss sobr gêner escol promet ajud municípi constru crech ' => Neutro [Neutro]
'grand culp aind sequ eleit sequ extingu orgã inútil dé dinh sufic

## SVM model

In [133]:
model_SVC = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(random_state=42)),
])

## SVM model with Hold out

In [ ]:
model_SVC.fit(X_train, y_train)
predictions_SVC = model_SVC.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions_SVC))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

    Negativo       0.00      0.00      0.00      9813
      Neutro       0.33      1.00      0.50      9664
    Positivo       0.00      0.00      0.00      9813

    accuracy                           0.33     29290
   macro avg       0.11      0.33      0.17     29290
weighted avg       0.11      0.33      0.16     29290



In [ ]:
for doc, prediction, y in zip(X_test[0:20], predictions_SVC[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'saib nazist ode jud jerusalém caracterís esquerdopat . além diss esquerd ador corrupt , pedófil , estupr marg . enquadr ness tim ? pesquis amig lulal merd mesm cois :-) ' => Positivo [Positivo]
'príncip harry complet 34 ano receb homen intern ' => Neutro [Neutro]
'blizzard problem overwatch , diz licenç jog tav jog ate ont boa , tod bug compr jog mó feliz mei deix entr ' => Positivo [Negativo]
'oi mig ' => Positivo [Positivo]
'imagin iri ter gent centr quer ir pro lad bolsonar . ' => Negativo [Positivo]
'ali marin tem alv dobr bolsonar daciol debat . ' => Negativo [Neutro]
'aha !!! import pro irm irm vai sent bem famíl lá ^^ ' => Positivo [Positivo]
'big broth ingl políci vê 11 mil hor víde ach suspeit envenen ex espiã . ' => Neutro [Neutro]
'agor sim united par cicl posi watford som prim derrot nest ediç premi leag ' => Positivo [Positivo]
'post elogi manu aqu . ler post , vai entend toler ser zer pra tip açã . ele terr . candidat pres . atitud infeliz . :-( ' => Negativo [Negativo]


## Logistic Regression

In [138]:
model_LR = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42, multi_class='multinomial', max_iter=500, n_jobs=-1)),
])

## Logistic Regression with Hold Out

In [139]:
model_LR.fit(X_train, y_train)
predictions_LR = model_LR.predict(X_test)

In [140]:
X_train.shape, X_test.shape

((56229,), (27695,))

In [141]:
print(classification_report(y_test, predictions_LR))

              precision    recall  f1-score   support

    Negativo       0.72      0.78      0.75      9227
      Neutro       0.85      0.86      0.86      9124
    Positivo       0.70      0.64      0.67      9344

    accuracy                           0.76     27695
   macro avg       0.76      0.76      0.76     27695
weighted avg       0.76      0.76      0.76     27695



In [142]:
for doc, prediction, y in zip(X_test[0:20], predictions_LR[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'prefir estud histór polít ' => Positivo [Positivo]
'sig volt nunc ped nad ' => Positivo [Positivo]
'gt tit diz aráb saudit vai oferec grau dificuldad mai brasil ' => Neutro [Neutro]
'fic cad dia preocup med merd sab ' => Negativo [Negativo]
'comcast torn mai acion emis tv sky ' => Neutro [Neutro]
'pasó pt ' => Negativo [Negativo]
'funcion mus nacion consegu retir algum document mater prédi ' => Neutro [Neutro]
'fal boz ach vai vir dit vot cir outr " prot neo dit merd " ' => Negativo [Positivo]
'vej soluç plaus urgent problem arm invest nad ferr enquant corrupç val pen est merc viol ' => Positivo [Negativo]
'pessoal arroi so olá nom catet ' => Positivo [Positivo]
'ent usa camiset ulstr filh put ' => Positivo [Positivo]
'gt govern recu dev prop adi reajust serv ' => Neutro [Neutro]
'cir amo ' => Negativo [Negativo]
'com log ' => Positivo [Positivo]
'film sobr pornochanch estre seman conf novidad tel ' => Neutro [Neutro]
'aproveit mat haddad lul cir fic tod mund feliz ' => Positivo [Posi

## Logistic Regression with cross-validation and gridsearch 
Doing CV and gridsearch on the model with the best performance with the default settings 

In [143]:
model_LR = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42, multi_class='multinomial', max_iter=500, n_jobs=-1)),
])

In [156]:
param_grid_LR = {
    'clf__solver': ['newton-cg', 'lbfgs'],
    'clf__penalty': ['l2', 'none'],
}

In [157]:
#grid_LR = GridSearchCV(param_grid=param_grid_LR, estimator=model_LR, scoring='accuracy', cv=5, verbose=1)

In [158]:
#grid_LR.fit(X_tweets, y_classes)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 53.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        prep

In [160]:
#print(grid_LR.best_params_)

{'clf__penalty': 'l2', 'clf__solver': 'newton-cg'}


In [161]:
model = LogisticRegression(random_state=42, multi_class='multinomial', max_iter=100, n_jobs=-1, solver= 'newton-cg')

In [162]:
model_LR = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
])

In [163]:
predictions_LR = cross_val_predict(model_LR, X_tweets, y_classes, cv = 5)

In [164]:
print(classification_report(y_classes, predictions_LR))

              precision    recall  f1-score   support

    Negativo       0.72      0.78      0.75     28000
      Neutro       0.85      0.84      0.85     27991
    Positivo       0.67      0.62      0.65     27933

    accuracy                           0.75     83924
   macro avg       0.75      0.75      0.75     83924
weighted avg       0.75      0.75      0.75     83924



In [165]:
for doc, prediction, y in zip(X_test[0:20], predictions_LR[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'prefir estud histór polít ' => Neutro [Positivo]
'sig volt nunc ped nad ' => Neutro [Positivo]
'gt tit diz aráb saudit vai oferec grau dificuldad mai brasil ' => Positivo [Neutro]
'fic cad dia preocup med merd sab ' => Neutro [Negativo]
'comcast torn mai acion emis tv sky ' => Neutro [Neutro]
'pasó pt ' => Positivo [Negativo]
'funcion mus nacion consegu retir algum document mater prédi ' => Neutro [Neutro]
'fal boz ach vai vir dit vot cir outr " prot neo dit merd " ' => Neutro [Positivo]
'vej soluç plaus urgent problem arm invest nad ferr enquant corrupç val pen est merc viol ' => Positivo [Negativo]
'pessoal arroi so olá nom catet ' => Neutro [Positivo]
'ent usa camiset ulstr filh put ' => Positivo [Positivo]
'gt govern recu dev prop adi reajust serv ' => Neutro [Neutro]
'cir amo ' => Negativo [Negativo]
'com log ' => Negativo [Positivo]
'film sobr pornochanch estre seman conf novidad tel ' => Neutro [Neutro]
'aproveit mat haddad lul cir fic tod mund feliz ' => Neutro [Positivo]
'ten

## Exporting models

In [166]:
import pickle

In [175]:
#filename = 'LR_model.sav'
pickle.dump(model_LR, open('LR_model_we', 'wb'))

In [181]:
loaded_model = pickle.load(open('LR_model_we', 'rb'))

In [182]:
loaded_model

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
   